# Modellbildung

In [ ]:
import sys
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
from IPython.display import display, Image
from scipy.optimize import curve_fit
from scipy.integrate import solve_ivp

In [ ]:
imag = Image("images/zweitank.png", width=600)
display(imag)

Ziel ist die Modellierung des Eingangs-Ausgangsverhaltens des bekannten 2-Tanksystems.
Durch die Modellierung aus der Übung ist bekannt, dass es sich bei dem linearisierten Modell um ein System zweiter Ordnung
\begin{align*}
    T_1T_2\ddot{h}_2(t) + (T_1 + T_2)\dot{h}_2(t) + h_2(t) & = K_\text{P} u(t)
\end{align*}
handelt.

Nachfolgend sollen die drei parameterierte Varianten des Modells ermittelt und simuliert werden:

1. auf Basis der physikalischen Parameter,
2. ausgehend von der aufgenommenen Sprungantwort für das Gesamtsystem sind die Parameter zu bestimmen sowie
3. Seperierung des Systems in zwei Einzeltanksysteme in Systeme erster Ordnung und Bestimmung der Parameter auf Basis der aufgenommenen Sprungangworten für die beiden Teilsysteme

### Messdaten einlesen

In [ ]:
messungTank1Tank2 = pd.read_csv('20221129_Sprungantwort_9V_Aufbau1_V12_2_0vU_V21_2_1_vU.csv')
messungTank2 = pd.read_csv('20221129_Sprungantwort_Aufbau1_Pumpe_in_Tank2_V21_2_1vU_9V.csv')

### 1. Physikalische Parameter

Bestimmen Sie die Parameter für das linearisierte System auf Basis der gegebenen physikalischen Parameter für die Ruhelage $\bar{u} = 9\, \mathrm{V}$.

In [ ]:
Kp_analytic = 0.1384482
T1_analytic = 32.993533
T2_analytic = 65.106089

display(Markdown(
   rf"""
$K_\text{{p}} = {Kp_analytic}$

$T_1 = {T1_analytic}$

$T_2 = {T2_analytic}$
"""))

pAnalytic = [Kp_analytic, T1_analytic, T2_analytic]

### 2. Parameter des messtechnisch erfassten PT$_2$-Elements

Um die Parameter zu bestimmen, wird die analytische Lösung des Systems zweiter Ordnung für einen Einheitssprung $\sigma$ als Eingang verwendet:
\begin{align*}
h(t) = K_\text{P}\left(1 - \frac{1}{T_1-T_2}\left(T_1 e^{-\frac{t}{T_1}} - T_2 e^{-\frac{t}{T_2}}\right)\right)
\end{align*}

Die Parameter $T_1$, $T_2$ und $K$ werden unter Verwendung der vorangegangenen Lösung mittels eines Optimierungsverfahrens bestimmt.

In [ ]:
def pt2Analytic(t, Kp, T1, T2):
    val = Kp * (1 - 1 / (T1 - T2) * (T1 * np.exp(-t/T1) - T2 * np.exp(-t/T2)))
    for i in range(len(val)):
        if not math.isfinite(val[i]):
            print(val[i])
            val[i] = 0
    return val

In [ ]:
xData = messungTank1Tank2['time']
yData = messungTank1Tank2['HeightT2']

# errorcorrect measured data (in HeightT2 some values are nan)
for i in range(len(yData)):
   if not math.isfinite(yData[i]):
      try:
         # if value is nan, use mean of prev and next value
         yData[i] = (yData[i-1] + yData[i+1]) /2
      except: 
         print('implement more ')

# Delete part of the first second in which the pump voltage ramps up
time = 0.2
for i in range(len(xData)):
   if xData[i] >= time:
      # delete first entries, adapt indexes, set starttime = 0
      xData = xData.iloc[i:]
      xData.index -= i
      xData -= time
      # also delete corresponding height entries
      yData = yData.iloc[i:]
      yData.index -= i
      break

pOpt, _ = curve_fit(pt2Analytic, xdata=xData, ydata=yData, p0=pAnalytic)

Kp_pt2 = pOpt[0]
T1_pt2 = min(pOpt[1], pOpt[2])
T2_pt2 = max(pOpt[1], pOpt[2])

display(Markdown(
   rf"""
$K_\text{{p}} = {Kp_pt2}$

$T_1 = {T1_pt2}$

$T_2 = {T2_pt2}$
"""))

### 3.  Parameter der messtechnisch erfassten seriell verschalteten PT$_1$-Elemente

Durch Aufspaltung des Systems in zwei Einzelelemente, die die Form eines Systems erster Ordnung annehmen, können durch Auswerten der Sprungantwort
\begin{align}
    h(t) & = K_i \left(1 - e^{-\frac{t}{T_i}}\right), & i & = 1,2
\end{align}
die Zeitkonstanten bestimmt werden.

In [ ]:
def pt1Analytic(t, Kp, T1):
    return Kp * (1 - np.exp(-t / T1))

#### Tank 1

In [ ]:
xDataT1 = messungTank1Tank2['time']
yDataT1 = messungTank1Tank2['HeightT1']

# curvefitting and printing the result
pOpt, _ = curve_fit(pt1Analytic, xdata=xDataT1, ydata=yDataT1)

# storing results
K1_pt1 = pOpt[0]
T1_1_pt1 = pOpt[1]

display(Markdown(
rf"""
$K_1 = {K1_pt1}$

$T_1 = {T1_1_pt1}$
"""))

#### Tank 2

In [ ]:
xDataT2 = messungTank2['time']
yDataT2 = messungTank2['HeightT2']

pOpt, _ = curve_fit(pt1Analytic, xdata=xDataT2, ydata=yDataT2)

# storing results
K_2 = pOpt[0]
T2_2_pt1 = pOpt[1]

display(Markdown(
   rf"""
$K_2 = {K_2}$

$T_2 = {T2_2_pt1}$
"""))

In [ ]:
Kp_pt1 = Kp_pt2
T1_pt1 = min(T1_1_pt1, T2_2_pt1)
T2_pt1 = max(T1_1_pt1, T2_2_pt1)

display(Markdown(
    rf"""
$K_\text{{p}} = {Kp_pt1}$

$T_1 = {T1_pt1}$

$T_2 = {T2_pt1}$
"""))

## Vergleich aller Modelle

In [ ]:
params = {
    'T1': [round(T1_analytic, 5), round(T1_pt2, 5), round(T1_pt1, 5)],
    'T2': [round(T2_analytic, 5), round(T2_pt2, 5), round(T2_pt1, 5)],
    'Kp': [round(Kp_analytic, 5), round(Kp_pt2, 5), round(Kp_pt1, 5)],
}
print ("{:<5} {:<11} {:<9} {:<11}".format(' ','analytisch','PT2','PT1'))
for name, values in params.items():
    s1, s2, s3 = values
    print ("{:<5} {:<11} {:<9} {:<11}".format(name, s1, s2, s3))

In [ ]:
def linSys(t, x, u, A, B):
    return A.dot(x) +  B.dot(u)

In [ ]:
A_analytic = np.array([[0, 1],
                       [-1 / (T1_analytic * T2_analytic), - (T1_analytic + T2_analytic) / (T1_analytic * T2_analytic)]])
B_analytic = np.array([[0],
                       [Kp_analytic / (T1_analytic * T2_analytic)]])

A_pt2 = np.array([[0, 1],
                  [-1 / (T1_pt2 * T2_pt2), - (T1_pt2 + T2_pt2) / (T1_pt2 * T2_pt2)]])
B_pt2 = np.array([[0],
                  [Kp_pt2 / (T1_pt2 * T2_pt2)]])

A_pt1 = np.array([[0, 1],
                  [-1 / (T1_pt1 * T2_pt1), - (T1_pt1 + T2_pt1) / (T1_pt1 * T2_pt1)]])
B_pt1 = np.array([[0],
                  [Kp_pt1 / (T1_pt1 * T2_pt1)]])

In [ ]:
timeDom = np.linspace(0, len(messungTank1Tank2['HeightT2']), len(messungTank1Tank2['HeightT2'])) / 10
x0 = [0, 0]
u = [1]

solAnalytic = solve_ivp(linSys, [timeDom[0], timeDom[-1]], x0, t_eval=timeDom, args=(u, A_analytic, B_analytic))
solPT2 = solve_ivp(linSys, [timeDom[0], timeDom[-1]], x0, t_eval=timeDom, args=(u, A_pt2, B_pt2))
solPT1 = solve_ivp(linSys, [timeDom[0], timeDom[-1]], x0, t_eval=timeDom, args=(u, A_pt1, B_pt1))

In [ ]:
plt.close()

fig1, axes10 = plt.subplots(1, 1, figsize=(12,7))

axes10.plot(messungTank1Tank2['time'], messungTank1Tank2['HeightT2'], label=r'Messung Tank 1 & Tank 2')
axes10.plot(messungTank2['time'], messungTank2['HeightT2'], label=r'Messung Tank 2')
axes10.plot(timeDom, solAnalytic.y[0], label='physikalisch')
axes10.plot(timeDom, solPT2.y[0], label=r'PT$_2$')
axes10.plot(timeDom, solPT1.y[0], label=r'PT$_1$')

axes10.set_ylabel(r'$z_2$ [$m$]')
axes10.set_xlabel(r'Zeit [$s$]')

handlesAx1, labelsAx1 = axes10.get_legend_handles_labels()
axes10.legend([handle for i, handle in enumerate(handlesAx1)],
              [label for i, label in enumerate(labelsAx1)],
              bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
              ncol=9, mode="expand", borderaxespad=0., framealpha=0.5)

axes10.grid()
plt.show()